In [1]:
import torch
import random
import torch.nn as nn
import numpy as np
import math

- Below defined the bitwidth for fixed_point data, **nbits** represents the whole bitwidth of a fixed_point number, while **frac_bits** states the bitwidth for a fractional part in a fixed_point number.

In [2]:
frac_bits = 8
nbits = 16

- Below are functions that I would need to **transfer** floating number to fixed data and **format** them into strings.

In [18]:
def int_to_hex(val, nbits):
    if val < 0:
        return hex((val + (1 << nbits)) % (1 << nbits))
    else:
        return "{0:#0{1}x}".format(val, 2 + int(nbits / 4))
        

def fixed_point_multiply(x,y,frac_bits=8):
    return int(x*y/(2**frac_bits))

def format_array_to_string(arr, vhdl_prefix=None, nbits=16):
    if vhdl_prefix is None:
        string_to_return = "X_MEM :="
    else:
        string_to_return = vhdl_prefix
    string_to_return += " ("

    for i in range(2**math.ceil(math.log2(len(arr)))):
        if i< len(arr):
            string_to_return += "x\""+int_to_hex(arr[i], nbits)[2:]+"\","
        else:
            string_to_return += "x\""+int_to_hex(0, nbits)[2:]+"\","
    string_to_return = string_to_return[:-1]  +");"

    return string_to_return

def float_array_to_int(float_array, frac_bits=8):
    scaled_array = float_array*2**frac_bits
    int_array = scaled_array.astype(np.int16)
    return int_array

def float_array_to_string(float_array, vhdl_prefix=None, frac_bits=8, nbits=16):
    scaled_array = float_array*2**frac_bits
    int_array = scaled_array.astype(np.int16)
    return format_array_to_string(int_array, vhdl_prefix, nbits)

def int_to_bit(val, nbits):
    format_str = "{0:0"+str(nbits)+"b}"
    if val < 0:
        return bin((val + (1 << nbits)) % (1 << nbits))[2:]
    else:
        return format_str.format(val)

def mem_array_to_dat_file(file_name="", float_arr=None, nbits=16):

    scaled_array = float_arr*2**frac_bits
    int_array = scaled_array.astype(np.int16)
    # file open
    if file_name=="":
        print("you must specify the file name where to dump your array.\r\n")
        return

    string_to_write = ""
    # if len(int_array) < 2**math.ceil(math.log2(len(int_array))):
    for i in range(2**math.ceil(math.log2(len(int_array)))):
        if i< len(int_array):
            string_to_write += (int_to_bit(int_array[i], nbits)+'\r')
        else:
            string_to_write += (int_to_bit(0, nbits)+'\r')

    txt_file = open(file_name, "w")
    txt_file.write(string_to_write[:-1])
    txt_file.close()

weight_ih_l[k] : `(W_ii|W_if|W_ig|W_io)`

weight_hh_l[k] : `(W_hi|W_hf|W_hg|W_ho)`

bias_ih_l[k] :  `(b_ii|b_if|b_ig|b_io)`

bias_hh_l[k] :  `(W_hi|W_hf|W_hg|b_ho)`

In [4]:
torch.manual_seed(0)
random.seed(0)
input_size  = 5
hidden_size = 20
len_weights = (input_size+hidden_size) * hidden_size
len_bias = hidden_size

lstm_singal_cell = nn.LSTMCell(input_size, hidden_size) # (input_size, hidden_size)

for name, param in lstm_singal_cell.named_parameters():
    
    if name == 'weight_ih':
        weight_ih = param.detach().numpy()
    elif name == 'weight_hh':
        weight_hh = param.detach().numpy()
    elif name == 'bias_ih':
        bias_ih = param.detach().numpy()
    elif name == 'bias_hh':
        bias_hh = param.detach().numpy()

weights= np.hstack((weight_ih,weight_hh)).flatten().flatten()
bias=bias_hh+bias_ih

Wi = weights[len_weights*0:len_weights*1] #[Wii, Whi]
Wf = weights[len_weights*1:len_weights*2] #[Wif, Whf]
Wg = weights[len_weights*2:len_weights*3] #[Wig, Whg]
Wo = weights[len_weights*3:len_weights*4] #[Wio, Who]

Bi = bias[len_bias*0:len_bias*1] #B_ii+B_hi
Bf = bias[len_bias*1:len_bias*2] #B_if+B_hf
Bg = bias[len_bias*2:len_bias*3] #B_ig+B_hg
Bo = bias[len_bias*3:len_bias*4] #B_io+B_ho

print(float_array_to_string(float_array=Wi, vhdl_prefix="signal ROM : WI_ARRAY :=", frac_bits=frac_bits, nbits=nbits))
print(float_array_to_string(float_array=Wf, vhdl_prefix="signal ROM : WF_ARRAY :=", frac_bits=frac_bits, nbits=nbits))
print(float_array_to_string(float_array=Wg, vhdl_prefix="signal ROM : WG_ARRAY :=", frac_bits=frac_bits, nbits=nbits))
print(float_array_to_string(float_array=Wo, vhdl_prefix="signal ROM : WO_ARRAY :=", frac_bits=frac_bits, nbits=nbits))

print(float_array_to_string(float_array=Bi, vhdl_prefix="signal ROM : BI_ARRAY:=", frac_bits=frac_bits, nbits=nbits))
print(float_array_to_string(float_array=Bf, vhdl_prefix="signal ROM : BF_ARRAY:=", frac_bits=frac_bits, nbits=nbits))
print(float_array_to_string(float_array=Bg, vhdl_prefix="signal ROM : BG_ARRAY:=", frac_bits=frac_bits, nbits=nbits))
print(float_array_to_string(float_array=Bo, vhdl_prefix="signal ROM : BO_ARRAY:=", frac_bits=frac_bits, nbits=nbits))

signal ROM : WI_ARRAY := (x"0000",x"001e",x"ffd1",x"ffd6",x"ffea",x"0024",x"0004",x"ffe5",x"0034",x"0017",x"ffd5",x"0036",x"002b",x"ffec",x"0020",x"ffe0",x"fff8",x"0030",x"0002",x"ffd8",x"ffed",x"fff1",x"fff5",x"0005",x"0034",x"000f",x"ffff",x"002d",x"fffb",x"000f",x"0003",x"ffdd",x"0002",x"001b",x"001c",x"ffcc",x"fff6",x"ffd6",x"ffe8",x"0014",x"ffd8",x"0015",x"0030",x"0003",x"ffda",x"ffec",x"000c",x"ffd5",x"001c",x"ffcd",x"ffef",x"fff5",x"ffca",x"ffdb",x"ffe9",x"ffc9",x"ffc9",x"fff5",x"0026",x"ffca",x"002f",x"ffea",x"0010",x"0002",x"ffcd",x"002f",x"001e",x"0038",x"001c",x"ffdb",x"002f",x"0003",x"001b",x"ffd3",x"fff0",x"0002",x"0016",x"0022",x"ffda",x"ffe8",x"ffc8",x"ffea",x"000c",x"ffd4",x"0012",x"001e",x"0007",x"ffda",x"ffd4",x"ffef",x"0019",x"0038",x"0020",x"fffa",x"0014",x"ffc8",x"ffd0",x"001a",x"ffe0",x"001b",x"0014",x"002f",x"fff5",x"002a",x"fff7",x"ffd8",x"ffe4",x"ffd1",x"001d",x"fffb",x"002c",x"0023",x"001f",x"0001",x"ffef",x"fff4",x"0007",x"001c",x"ffd8",x"0030",x"fffa",x"ffd1

In [21]:
torch.manual_seed(0)
random.seed(0)

input = torch.randn(2, 1, 5)  # (time_steps, batch, input_size)
hx = torch.randn(1, 20)  # (batch, hidden_size), this is the hidden states
cx = torch.randn(1, 20)  # this the cell states
# input = input/torch.max(abs(input))
# hx = hx/torch.max(abs(hx))
# cx = cx/torch.max(abs(cx))
output = []
for i in range(input.size()[0]):
    print('===========round%s==========='%str(i))
    # print("input_x",input[i])
    # print("h_x",hx)
    # print("c_x",cx)
    x_h_input = np.hstack((input[i].detach().numpy().flatten(),hx.detach().numpy().flatten()))
    print(float_array_to_string(x_h_input, vhdl_prefix="signal test_x_h_data : X_H_ARRAY := ", frac_bits=frac_bits, nbits=nbits))
    print(float_array_to_string(cx.detach().numpy().flatten(), vhdl_prefix="signal test_c_data : C_ARRAY := ", frac_bits=frac_bits, nbits=nbits))

    hx, cx = lstm_singal_cell(input[i], (hx, cx))
    # print("h_x out:", hx)
    # print("c_x out:", cx)
    print("h_out:")
    print(float_array_to_int(hx.detach().numpy().flatten(), frac_bits=frac_bits))
    output.append(hx)

===========round0===========
signal test_x_h_data : X_H_ARRAY :=  (x"018a",x"ffb5",x"fdd3",x"0091",x"feeb",x"0099",x"fe72",x"ffa9",x"01da",x"ffc9",x"ff42",x"0090",x"0042",x"ffd4",x"ff53",x"00f0",x"007d",x"0134",x"0015",x"fecd",x"ffff",x"ff7c",x"ffb2",x"fe6c",x"01b4",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000");
signal test_c_data : C_ARRAY :=  (x"0034",x"ff8d",x"ff6e",x"ff72",x"fee0",x"ffaf",x"fee9",x"ffeb",x"ffe9",x"00af",x"ff2a",x"0000",x"ff40",x"002f",x"009f",x"00a3",x"ffc2",x"024d",x"fe1f",x"fff4",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000",x"0000");
h_out:
[  34  -80  -32  -28  -88   11  -60    6  -16   18  -32   46  -77   15
   70   27   13  112 -156    3]
===========round1===========
signal test_x_h_data : X_H_ARRAY :=  (x"fe9a",x"0067",x"00d6",x"ff48",x"ff99",x"0022",x"ffb0",x"ffe0",x"ffe4",x"ffa8",x"000b",x"ffc4",x"0006",x"fff0",x"0012",x"ffe0",x"002e",x"ffb3",x"000f",x"0046",x"001b",x"000d",x"0070",x"ff64",x"0003",x"0000